# EDA

In [14]:
import pandas as pd
from datetime import datetime

In [12]:
coop_data = "data_raw/cooperatives_data.xlsx"
log_file = "xlsx_to_csv_log.txt"

CONVERT XLXS TO CSV FOR BETTER TREATMENT

In [15]:
with open(log_file, "w", encoding="utf-8") as log:
    log.write("=== LOG DE CONVERSION EXCEL → CSV ===\n")
    log.write(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log.write(f"Fichier source : {coop_data}\n\n")

    # Charger toutes les feuilles
    sheets = pd.read_excel(coop_data, sheet_name=None)

    for sheet_name, df in sheets.items():
        csv_name = f"{sheet_name}.csv"

        # Sauvegarder la feuille
        df.to_csv(csv_name, index=False, encoding="utf-8")

        # Recharger le CSV pour vérification
        df_check = pd.read_csv(csv_name)

        # Vérification d’intégrité
        rows_before, cols_before = df.shape
        rows_after, cols_after = df_check.shape

        ok = (rows_before == rows_after) and (cols_before == cols_after)
        status = "✅ OK" if ok else "⚠️ MISMATCH"

        # Écrire dans le log
        log.write(f"Feuille : {sheet_name}\n")
        log.write(f" - CSV : {csv_name}\n")
        log.write(f" - Lignes (Excel → CSV) : {rows_before} → {rows_after}\n")
        log.write(f" - Colonnes (Excel → CSV) : {cols_before} → {cols_after}\n")
        log.write(f" - Statut : {status}\n\n")

        print(f"{status} - Feuille '{sheet_name}' convertie en '{csv_name}'")

    log.write("=== FIN DU TRAITEMENT ===\n")

✅ OK - Feuille '(1) Registre Producteurs' convertie en '(1) Registre Producteurs.csv'
✅ OK - Feuille '(2) Registre Plantations' convertie en '(2) Registre Plantations.csv'


# Contrôle Qualité – Données Coopératives Cacao

## 1. Premier apperçu des fichiers

### **CSV**
- En-têtes longues, caractères spéciaux, retours à la ligne → à **normaliser**.  
- Valeurs `"Non Disponible"` → à **standardiser en `NaN`**.  
- Champs numériques *(surfaces, lat/lon)* → possibles **séparateurs décimaux “,”** à convertir en points.  

### **GeoJSON**
- `crs = OGC:CRS84` → correspond à **WGS84 (lon/lat)** OK  
- `geometry = MultiPolygon` → on **recalculera l’aire**.  
- Propriétés `Latitude` / `Longitude` utilisent des **virgules** (“6,3796…”, “-5,1983…”) alors que la **géométrie est correcte** avec des points →  
  **on se fie uniquement à la géométrie** pour le spatial.  

### **Clés d’appariement pressenties**
- `Farms_ID` ~ `Code plantation` *(ex : `1030040479-P001` ~ …-P001)*  
- `Farmer_ID` ~ `Code Producteur` *(ex : `1030040479`)*  

---

## 2. Standards du projet (clé pour la suite)

### Schéma cible – Colonnes “propres”

#### **Plantations** (`coop_plantations.csv`)
| Nom colonne | Description / origine |
|--------------|----------------------|
| `plantation_id` | ex-**Code plantation*** |
| `producer_id` | ex-**Code Producteur*** |
| `coop` | ex-**Cooperative / Groupe*** |
| `area_declared_ha` | ex-**Superficie Cacao (Ha)*** |
| `yield_kg_per_ha` | ex-**Rendement (kg/Ha)*** |
| `estimation_kg` | ex-**Estimation (Kg)*** |
| `variete` | ex-**Variété Cacao** |
| `has_manager` | **Oui / Non / Inconnu** |
| `manager_name` | Nom du gestionnaire |
| `lat_declared`, `lon_declared` | Coordonnées déclarées |

---

#### **Producteurs** (`coop_producteurs.csv`)
| Nom colonne | Description / origine |
|--------------|----------------------|
| `producer_id` | ex-**Code Producteur*** |
| `coop`, `cert_codes` | Informations coopérative / certifications |
| `birth_year`, `join_date_mmyy`, `gender` | Informations personnelles |
| `surface_totale_exploitation_ha`, `surface_cacao_totale_ha` | Surfaces totales |
| `nb_plantations_cacao` | Nombre de plantations cacao |
| `estimation_totale_kg` | Estimation totale annuelle |
| `ville`, `coach_name`, `matricule`, `matricule_type` | Données administratives |
| `household_size` | Taille du ménage |
| `recolte_2425_kg`, `livre_coop_2425_kg` | Données campagne 2024-2025 |
| `recolte_2324_kg`, `livre_coop_2324_kg` | Données campagne 2023-2024 |

---

#### **Parcelles** (`parcelles.geojson`)
| Nom colonne | Description / origine |
|--------------|----------------------|
| `plantation_id` | map depuis **Farms_ID** |
| `producer_id` | map depuis **Farmer_ID** |
| `area_geo_ha` | **Aire recalculée** depuis la géométrie projetée |
| `geometry` | **MultiPolygon** |
| *(optionnel)* `lat_geo`, `lon_geo` | **Centroïde** de la géométrie |

---

### Calcul de surface
- **Projection recommandée :** `UTM 30N – EPSG:32630` *(Côte d’Ivoire, long ≈ −5)*  
- **Formule :**
aire_ha = aire_m2 / 10_000
